# Investing On ETFs Using The Kelly Formula Part 2

## How To Allocate Capital Using The Kelly Formula

## Introduction
...

## The Kelly Formula

### Ed Thorp
...

### Optimal Capital Allocation
...

## Menu
1. ...
2. ...

Import packages

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

Set group keys

In [2]:
groups = ['us bonds',
          'us stocks',
          'intl bonds',
          'intl stocks',
          'sectors']

Set input input files

In [3]:
input = {'us bonds': 'etf_us_bonds.pickle', 
         'us stocks': 'etf_us_stocks.pickle',
         'intl bonds': 'etf_intl_bonds.pickle',
         'intl stocks': 'etf_intl_stocks.pickle',
         'sectors': 'etf_sectors.pickle'}

Create output dictionary

In [4]:
output = {'us bonds': {},
          'us stocks': {},
          'intl bonds': {},
          'intl stocks': {},
          'sectors': {}}

Set market parameters:
1. Risk-free rate (yearly)
2. Maximum leverage allowed by US-regulated brokers

In [5]:
# Paremeters
risk_free = 0.025
max_leverage = 4.00

Calculate Kelly leverage of each ETF group

In [6]:
for i in groups:    

    # Load file
    with open(input[i], 'rb') as f:
        close = pickle.load(f)
    f.close()
    
    # Daily returns
    returns = close.pct_change()
    
    # Excess daily returns
    excess_returns = returns - risk_free / 250
    
    # Mean excess daily returns annualized
    M = excess_returns.mean() * 250
    
    # Covariance of daily returns
    C = returns.cov() * 250
    
    # Kelly leverage: F = C^-1 * M
    F = np.matmul(np.linalg.inv(C), M)
    
    # Constraint of max leverage 4:1
    mask = (F > max_leverage)
    F[mask] = max_leverage
    mask = (F < -max_leverage)
    F[mask] = - max_leverage
    
    # Half Kelly 
    F2 = 0.5 * F
    
    # Growth rate
    g = risk_free + np.matmul(F, np.transpose(M)) - 0.5 * np.matmul(np.matmul(np.transpose(F), C), F)

    # Growth at half Kelly
    g2 = risk_free + np.matmul(F2, np.transpose(M)) - 0.5 * np.matmul(np.matmul(np.transpose(F2), C), F2)
    
    # Sharpe ratio
    sharpe = np.matmul(np.matmul(np.transpose(F), C), F)
    
    # Sharpe at half Kelly
    sharpe2 = np.matmul(np.matmul(np.transpose(F2), C), F2)
    
    # Update output
    output[i]['tickers'] = list(close.columns)
    output[i]['kelly'] = list(F)
    output[i]['kelly2'] = list(F2)
    output[i]['growth'] = g
    output[i]['growth2'] = g2
    output[i]['sharpe'] = sharpe
    output[i]['sharpe2'] = sharpe2

### Vanguard US Bonds ETFs

The Kelly leverage for US Bonds ETFs are

In [7]:
group = 'us bonds'

pd.DataFrame({'Kelly Leverage': output[group]['kelly'],
              'Half-Kelly Leverage': output[group]['kelly2']},
             index=output[group]['tickers'])

,Kelly Leverage,Half-Kelly Leverage
EDV,3.633497,1.816749
BIV,4.000000,2.000000
VGIT,4.000000,2.000000
BLV,-4.000000,-2.000000
VGLT,-4.000000,-2.000000
VMBS,-4.000000,-2.000000
BSV,-4.000000,-2.000000
VTIP,-4.000000,-2.000000
VGSH,-4.000000,-2.000000
BND,4.000000,2.000000


The growth rate and Sharpe ratio are

In [8]:
pd.DataFrame({'Kelly Leverage': [output[group]['growth'], output[group]['sharpe']],
              'Half-Kelly Leverage': [output[group]['growth2'], output[group]['sharpe2']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Kelly Leverage,Half-Kelly Leverage
Growth Rate,0.093937,0.067602
Sharpe Ratio,0.065067,0.016267


### Vanguard US Stocks ETFs

The Kelly leverage for US Stocks ETFs are

In [9]:
group = 'us stocks'

pd.DataFrame({'Kelly Leverage': output[group]['kelly'],
              'Half-Kelly Leverage': output[group]['kelly2']},
             index=output[group]['tickers'])

,Kelly Leverage,Half-Kelly Leverage
VIG,4.000000,2.000000
VUG,-4.000000,-2.000000
VYM,-3.070013,-1.535006
VV,-4.000000,-2.000000
MGC,1.691192,0.845596
MGK,4.000000,2.000000
MGV,-1.013273,-0.506637
VOO,0.841784,0.420892
VTI,1.814237,0.907118
VTV,4.000000,2.000000


The growth rate and Sharpe ratio are

In [10]:
pd.DataFrame({'Kelly Leverage': [output[group]['growth'], output[group]['sharpe']],
              'Half-Kelly Leverage': [output[group]['growth2'], output[group]['sharpe2']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Kelly Leverage,Half-Kelly Leverage
Growth Rate,0.263775,0.199656
Sharpe Ratio,0.442146,0.110537


### Vanguard International Bonds ETFs

The Kelly leverage for International Bonds ETFs are

In [11]:
group = 'intl bonds'

pd.DataFrame({'Kelly Leverage': output[group]['kelly'],
              'Half-Kelly Leverage': output[group]['kelly2']},
             index=output[group]['tickers'])

,Kelly Leverage,Half-Kelly Leverage
BNDX,4.0,2.0
VWOB,4.0,2.0


The growth rate and Sharpe ratio are

In [12]:
pd.DataFrame({'Kelly Leverage': [output[group]['growth'], output[group]['sharpe']],
              'Half-Kelly Leverage': [output[group]['growth2'], output[group]['sharpe2']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Kelly Leverage,Half-Kelly Leverage
Growth Rate,0.121379,0.080773
Sharpe Ratio,0.060666,0.015167


### Vanguard International Stocks ETFs

The Kelly leverage for International Stocks ETFs are

In [13]:
group = 'intl stocks'

pd.DataFrame({'Kelly Leverage': output[group]['kelly'],
              'Half-Kelly Leverage': output[group]['kelly2']},
             index=output[group]['tickers'])

,Kelly Leverage,Half-Kelly Leverage
VT,4.000000,2.000000
VEU,-4.000000,-2.000000
VSS,-4.000000,-2.000000
VEA,-2.112510,-1.056255
VGK,-0.206509,-0.103254
VPL,1.699139,0.849570
VNQI,4.000000,2.000000
VXUS,-4.000000,-2.000000
VWO,-0.971531,-0.485765


The growth rate and Sharpe ratio are

In [14]:
pd.DataFrame({'Kelly Leverage': [output[group]['growth'], output[group]['sharpe']],
              'Half-Kelly Leverage': [output[group]['growth2'], output[group]['sharpe2']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Kelly Leverage,Half-Kelly Leverage
Growth Rate,-0.217473,0.034939
Sharpe Ratio,1.049404,0.262351


### Vanguard Sectors ETFs

The Kelly leverage for sectors ETFs are

In [15]:
group = 'sectors'

pd.DataFrame({'Kelly Leverage': output[group]['kelly'],
              'Half-Kelly Leverage': output[group]['kelly2']},
             index=output[group]['tickers'])

,Kelly Leverage,Half-Kelly Leverage
VOX,-4.000000,-2.000000
VCR,2.802745,1.401372
VDC,-1.582406,-0.791203
VDE,-4.000000,-2.000000
VFH,4.000000,2.000000
VHT,-0.114157,-0.057078
VIS,-2.298666,-1.149333
VGT,4.000000,2.000000
VAW,-1.106886,-0.553443
VNQ,-2.011771,-1.005885


The growth rate and Sharpe ratio are

In [16]:
pd.DataFrame({'Kelly Leverage': [output[group]['growth'], output[group]['sharpe']],
              'Half-Kelly Leverage': [output[group]['growth2'], output[group]['sharpe2']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Kelly Leverage,Half-Kelly Leverage
Growth Rate,0.907456,0.602755
Sharpe Ratio,1.092213,0.273053


## Conclusion
...